# List DataSources

These instructions list the active datasources.

## Prepare Go enviroment

Define global imports, types, and variables.

In [ ]:
import (
	"google.golang.org/grpc"
	"google.golang.org/grpc/credentials/insecure"
    "github.com/senzing-garage/sz-sdk-go-grpc/szabstractfactory"
    "github.com/senzing-garage/sz-sdk-go/senzing"    
)

var (
    ctx = context.TODO()
	grpcAddress    = "localhost:8261"
)    

## Define functions

Create a function for testing error conditions.

In [ ]:
func testErr(err error) {
    if err != nil {
        panic(err)
    }
}

Create a function to pretty print JSON.

In [ ]:
func asPrettyJSON(str string) string {
	var prettyJSON bytes.Buffer
	if err := json.Indent(&prettyJSON, []byte(str), "", "    "); err != nil {
		return str
	}
	return prettyJSON.String()
}

## Main

Get the default configuration and pretty print the datasources.

In [ ]:
%%

// Create a Senzing AbstractFactory.

grpcConnection, err := grpc.NewClient(grpcAddress, grpc.WithTransportCredentials(insecure.NewCredentials()))
testErr(err)
szAbstractFactory := &szabstractfactory.Szabstractfactory{
    GrpcConnection: grpcConnection,
}  

// Create Senzing objects.

szConfig, err := szAbstractFactory.CreateConfig(ctx)
testErr(err)

szConfigManager, err := szAbstractFactory.CreateConfigManager(ctx)
testErr(err)

// Get Default configuration.

defaultConfigID, err := szConfigManager.GetDefaultConfigID(ctx)
testErr(err)

configDefinition, err := szConfigManager.GetConfig(ctx, defaultConfigID)
testErr(err)

configHandle, err := szConfig.ImportConfig(ctx, configDefinition)
testErr(err)

// Get datasources from Senzing configuration.

dataSources, err := szConfig.GetDataSources(ctx, configHandle)
testErr(err)

fmt.Println(asPrettyJSON(dataSources))